# Train Word2Vec

In [1]:
import codecs
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Dense,
    Input,
    LSTM,
    Embedding,
    Dropout,
    Activation,
    SpatialDropout1D
)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from ast import literal_eval
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/Users/sebastiancorrea/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('../data/pandas_data_frame.csv', index_col=0)
all_data = df.where((pd.notnull(df)), '')
all_data['hashtag'] = all_data['hashtag'].apply(literal_eval)

full_text = all_data['tidy_tweet'][(all_data['label']==1.0) | (all_data['label']==0.0)]
y = all_data['label'][(all_data['label']==1.0) | (all_data['label']==0.0)]

## Load word2vec matrix

In [ ]:
#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('../data/vectors/cc.en.300.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

In [ ]:
tk = Tokenizer(lower=True, filters='')
tk.fit_on_texts(full_text)

train_tokenized = tk.texts_to_sequences(full_text)
word_index = tk.word_index

X = pad_sequences(train_tokenized, maxlen=max_len)

x_train, x_val, y_train, y_val = train_test_split(X, y, random_state=1992, test_size=0.2)

print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

In [ ]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
import sys
sys.path.append("../") 
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1
from personal_library.sce_keras.callbacks import (
    LearningRateDecay,
    WarmUpCosineDecayScheduler
)


num_classes = 1
batch_size = 32
epochs = 200
learnRate = 0.001
lstm_out = 200
warmup_epoch = 20

lrate_decay = LearningRateDecay(epochs, learnRate).step_decay
warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learnRate,
                                        warmup_learning_rate=0,
                                        warmup_epoch=warmup_epoch,
                                        hold_base_rate_steps=5,
                                        verbose=0)

checkpoint_path = "../model_wehigts/6_w.hdf5"
checkpoint_path1 = "../model_wehigts/6_ch.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=2,
                               save_best_only=True, mode='min')
checkpointer1 = ModelCheckpoint(filepath=checkpoint_path1,
                               monitor='val_loss', verbose=2,
                               save_best_only=False, mode='min')

inp = Input(shape = (max_len,))
x = Embedding(nb_words+1, embed_size, weights = [embedding_matrix], trainable=False)(inp)
x = SpatialDropout1D(0.3)(x)
x = LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.5)(x)
x = Dense(1, activation = "sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.summary()

# 'binary_crossentropy'
model.compile(loss=f1_loss, 
              optimizer='adam', 
              metrics=['accuracy', f1]) 

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpointer, checkpointer1, lrate_decay])

In [ ]:
#Load best model
model.load_weights(checkpoint_path)
y_pred = model.predict(x_val, batch_size=1)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("Neural Network f1_sklearn: {}".format(f1_score(y_val, y_pred)))